This is the optimization code for computing our enumeration based Time-memory Trade-off curves. To run the code we have to execute the below two cells first.

In [1]:
import multiprocessing
multiprocessing.set_start_method('fork')

In [2]:
import collections
import random
from random import  uniform
from math import *
import scipy.optimize as opt
import math 
import matplotlib.pyplot as plt

def H(c):
    """
    Entropy function
    """
    if c == 0. or c == 1.:
        return 0.
    
    if c < 0. or c > 1.:
        return -1000
    
    return -(c * log2(c) + (1 - c) * log2(1 - c))

def binomH(n,k):
    """
    binomial coefficient
    """
    # if k/n not in ZZ:
    #     return -100
    if(n<=0):
        return 0.
    return n * H(k/n)

def multiH(n,c):
    """
    multinomial coefficient
    """
    if sum(c)>n:
        return 0
    tot=0
    val=n
    for i in c:
        tot+=binomH(n,i)
        n-=i
    return tot


def wrap(f,g) :
    def inner(x):
        return f(g(*x))
    return inner

def r(x,y,z):
    return [(ru(x,y)) for i in range(z)]

def reps1(w1,w0,omid,zmid2,zmid1,p): 
   #length of whole vec = p 
    if w1 <= 0.000001 or p == 0.:
        return 0
#     if p < w1 or p - 2*w1 < 2*zmid1:
#         return 0.
    
    return 2*multiH(w1, [omid,omid,w1*.5-omid]) + multiH(p-2*w1,[zmid2,zmid2,zmid1,zmid1])


def reps2(n1,n0,n2,tbar,o,z2,z1,p): 
    '''
    
    We represent a vector of length p having n1 many pm1,
    n2 many pm2 and rest zero, using rep-2 representation
    from our CRYPTO 2023 submission.
    
    '''
    
    if n1 < 0.000001 or p == 0.:
        return 0.
#     if p < n1 or p<n2 or p - 2*n1 - n2 < 2*z1:
#         return 0.
    
    return 2*multiH(n2,[tbar,tbar])+2*multiH(n1, [o, o, n1*.5-o])+multiH(n0, [z2, z2, z1, z1])

In [20]:
#Depth-4
# Defining the magic sequence
def c(i):
    return (i*i+3*i+4)/2

# Memory-complexity for dissection
def memory_dissect(search_space,i):
    return search_space/c(i)
# Time-complexity for dissection
def time_dissect(search_space, i):
    return c(i-1)*search_space/c(i)


set_vars = collections.namedtuple('LWE', ' g a l_1 l_2 l_3 omid zmid2 zmid1 tbar_1 o_1 z2_1 z1_1 tbar_2 o_2 z2_2 z1_2 ')
'''
tbar1  = (n2_1-t)/2
This set of variables can be further increased to 'tbar_3 o_3 z2_3 z1_3' to accomodate
more depth.
'''
def lwe(f) : return wrap(f, set_vars)

n2_0 = lambda x : 0
n1_0 = lambda x : x.g*w/2
n0_0 = lambda x : x.g*1 - 2*n1_0(x)

n2_1 = lambda x : x.omid+x.zmid2              
n1_1 = lambda x : n1_0(x)/2+x.zmid1           
n0_1 = lambda x : x.g*1-n1_0(x)-2*(x.zmid1+x.omid+x.zmid2)   

n2_2 = lambda x : x.tbar_1+x.o_1+x.z2_1
n1_2 = lambda x : n1_1(x)/2+x.z1_1+n2_1(x)-2*x.tbar_1
n0_2 = lambda x : x.g*1-n1_1(x)-2*n2_1(x)+2*(x.tbar_1-x.z1_1-x.o_1-x.z2_1)

n2_3 = lambda x : x.tbar_2+x.o_2+x.z2_2
n1_3 = lambda x : n1_2(x)/2+x.z1_2+n2_2(x)-2*x.tbar_2
n0_3 = lambda x : x.g*1-n1_2(x)-2*n2_2(x)+2*(x.tbar_2-x.z1_2-x.o_2-x.z2_2)


R_1 = lambda x : reps1(n1_0(x),n0_0(x),x.omid,x.zmid2,x.zmid1,x.g*1)
R_2 = lambda x : reps2(n1_1(x),n0_1(x),n2_1(x),x.tbar_1,x.o_1,x.z2_1,x.z1_1,x.g*1)


#search-space size in level-0
S_0 = lambda x : multiH(1,[n2_0(x),n2_0(x),n1_0(x),n1_0(x)])
S_1 = lambda x : multiH(x.g*1,[n2_1(x),n2_1(x),n1_1(x),n1_1(x)]) + multiH((1-x.g)/2,[(1-x.g)*w/4,(1-x.g)*w/4])
S_2 = lambda x : multiH(x.g*1,[n2_2(x),n2_2(x),n1_2(x),n1_2(x)]) + multiH((1-x.g)/4,[(1-x.g)*w/8,(1-x.g)*w/8])
SS_3 = lambda x : multiH(x.g*1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)]) + multiH((1-x.g)/8,[(1-x.g)*w/16,(1-x.g)*w/16])

q = lambda x : SS_3(x)*(x.a/c(suitable_diss(mem,x))-1/c(suitable_diss(mem,x)))

R_3 = lambda x : reps2(n1_2(x),n0_2(x),n2_2(x),x.tbar_2,x.o_2,x.z2_2,x.z1_2,x.g*1)+2*c(suitable_diss(mem,x))*q(x)

S_3 = lambda x : (multiH(x.g*1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)]) + multiH((1-x.g)/8,[(1-x.g)*w/16,(1-x.g)*w/16]))*x.a
S_mitm = lambda x : S_3(x)/c(suitable_diss(mem,x))

#probability that a random sum of two elements from level-1 forms a representation of any element from level-0
q_0 = lambda x : S_0(x)+R_1(x)-2*S_1(x)
q_1 = lambda x : S_1(x)+R_2(x)-2*S_2(x)
q_2 = lambda x : S_2(x)+R_3(x)-2*S_3(x)
q_3 = lambda x : 0

#unfiltered lists
L_mitm = lambda x : S_3(x)/2
L_3 = lambda x : 2*L_mitm(x)-x.l_3
L_2 = lambda x : 2*L_3(x)+2*q_3(x)-x.l_2
L_1 = lambda x : 2*L_2(x)+2*q_2(x)-x.l_1
L_0 = lambda x : 2*L_1(x)+2*q_1(x)-1

#filtered lists (for computing memory)
FL_3 = lambda x : q_3(x)+L_3(x)
FL_2 = lambda x : q_2(x)+L_2(x)
FL_1 = lambda x : q_1(x)+L_1(x)
FL_0 = lambda x : q_0(x)+L_0(x)

#choose magic sequence
def suitable_diss(mem,x):
    j = 0
    while S_3(x) / c(j) > mem(x):
        j = j+1
    return j

#memory for general dissection
def memdiss(x):
    x=set_vars(*x)
    i = suitable_diss(mem,x)
    mem1 = max(FL_3(x),FL_2(x),FL_1(x),FL_0(x),S_3(x)/c(i)) 
    return mem1

#time for general dissection
def timediss(x):
    x=set_vars(*x)
    delta = memdiss(x)/S_3(x)
    i = suitable_diss(mem,x)
    ti = max((i-c(i-1)*delta+2*delta)/(i+1),.5)
    ti = ti*S_3(x)
    return ti


def t_3(x):
    x=set_vars(*x)
    t3 = max(7*x.l_3-R_1(x)-2*R_2(x)-4*R_3(x),0)
    return t3

def t_2(x):
    x=set_vars(*x)
    t2 = max(7*x.l_3+3*x.l_2-R_1(x)-2*R_2(x)-4*R_3(x),0)-t_3(x)
    return t2

def t_1(x):
    x=set_vars(*x)
    t1 = max(7*x.l_3+3*x.l_2+x.l_1-R_1(x)-2*R_2(x)-4*R_3(x),0)-t_3(x)-t_2(x)
    return t1



def T_3(x):
    x=set_vars(*x)
    max_of_L_i = max(L_3(x),L_2(x),L_1(x),L_0(x),timediss(x))
    return max_of_L_i

def T_2(x):
    x=set_vars(*x)
    max_of_L_i = max(q_3(x)+L_3(x),L_2(x),L_1(x),L_0(x))
    return max_of_L_i

def T_1(x):
    x=set_vars(*x)
    max_of_L_i = max(q_2(x)+L_2(x),L_1(x),L_0(x))
    return max_of_L_i


mem = lambda x : 1

def time(x):
    x=set_vars(*x)
    max_of_L_i = max(t_1(x)+t_2(x)+t_3(x)+T_1(x),t_2(x)+t_3(x)+T_2(x),t_3(x)+T_3(x))
    return max_of_L_i

def memory(x):
    x=set_vars(*x)
    max_of_FL_i = max(memdiss(x),FL_3(x),FL_2(x),FL_1(x),FL_0(x)) #interpolate
    return max_of_FL_i

###########################################

constraints_lwe = [
    
        
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -memory(x)+mem(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_3(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_3(x))},


    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_0(x)*.5-x.omid)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+n0_0(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_1+n2_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_1+n1_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_1-x.z1_1+n0_1(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_2+n2_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_2+n1_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_2-x.z1_2+n0_2(x)/2)},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : x.l_3-R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : 3*x.l_3+x.l_2-2*R_3(x)-R_2(x))},
    

]

F6 = []
Data = []
    
for weight in [.5]:
    w = weight
    for M in range(0,1):
        global mem
        # Specify the ammount of available memory in the line below.
        mem = lambda x : .3426 #.02*(20-M)    
        res = 100
        ress = 5000
        for j in range(50):
            start=[uniform(1,1)]+[uniform(1,1)]+[uniform(0,.3)]*3+[uniform(0,.009)]*11

            bounds=[(1,1)]+[(1,1)]+[(0,1)]*3+[(0.,0.2)]*11
            result = opt.minimize(time, start, 
                    bounds= bounds, tol=1e-8, 
                    constraints=constraints_lwe, options={'maxiter':2000})
            r = result.get('fun')
            if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
                   res = r
                   ress = result
        if(res<memory(ress.x)):
            print('Memory exceeds time')
        print((mem(ress.x),round(res,4))) 
        F6.append((round(res,4),mem(ress.x)))
        Data.append(ress.x)
        #print(ress.x)
        print('*******************************')

/tmp/ipykernel_11084/1714754723.py:92: RuntimeWarning: divide by zero encountered in double_scalars
  delta = memdiss(x)/S_3(x)
/tmp/ipykernel_11084/1714754723.py:94: RuntimeWarning: invalid value encountered in double_scalars
  ti = max((i-c(i-1)*delta+2*delta)/(i+1),.5)


(0.3426, 0.3426)
*******************************


In [14]:
#depth-6 parallel

# Depth-6
# Defining the magic sequence
def c(i):
    return (i*i+3*i+4)/2

# Memory-complexity for dissection
def memory_dissect(search_space,i):
    return search_space/c(i)
# Time-complexity for dissection
def time_dissect(search_space, i):
    return c(i-1)*search_space/c(i)


set_vars = collections.namedtuple('LWE', ' g a l_1 l_2 l_3 l_4 l_5 omid zmid2 zmid1 tbar_1 o_1 z2_1 z1_1 tbar_2 o_2 z2_2 z1_2 tbar_3 o_3 z2_3 z1_3 tbar_4 o_4 z2_4 z1_4 ')
'''
tbar1  = (n2_1-t)/2
This set of variables can be further increased to 'tbar_3 o_3 z2_3 z1_3' to accomodate
more depth.
'''
def lwe(f) : return wrap(f, set_vars)



n2_0 = lambda x : 0
n1_0 = lambda x : x.g*w/2
n0_0 = lambda x : x.g*1 - 2*n1_0(x)

n2_1 = lambda x : x.omid+x.zmid2              
n1_1 = lambda x : n1_0(x)/2+x.zmid1           
n0_1 = lambda x : x.g*1-n1_0(x)-2*(x.zmid1+x.omid+x.zmid2)   

n2_2 = lambda x : x.tbar_1+x.o_1+x.z2_1
n1_2 = lambda x : n1_1(x)/2+x.z1_1+n2_1(x)-2*x.tbar_1
n0_2 = lambda x : x.g*1-n1_1(x)-2*n2_1(x)+2*(x.tbar_1-x.z1_1-x.o_1-x.z2_1)

n2_3 = lambda x : x.tbar_2+x.o_2+x.z2_2
n1_3 = lambda x : n1_2(x)/2+x.z1_2+n2_2(x)-2*x.tbar_2
n0_3 = lambda x : x.g*1-n1_2(x)-2*n2_2(x)+2*(x.tbar_2-x.z1_2-x.o_2-x.z2_2)

n2_4 = lambda x : x.tbar_3+x.o_3+x.z2_3
n1_4 = lambda x : n1_3(x)/2+x.z1_3+n2_3(x)-2*x.tbar_3
n0_4 = lambda x : x.g*1-n1_3(x)-2*n2_3(x)+2*(x.tbar_3-x.z1_3-x.o_3-x.z2_3)

n2_5 = lambda x : x.tbar_4+x.o_4+x.z2_4
n1_5 = lambda x : n1_4(x)/2+x.z1_4+n2_4(x)-2*x.tbar_4
n0_5 = lambda x : x.g*1-n1_4(x)-2*n2_4(x)+2*(x.tbar_4-x.z1_4-x.o_4-x.z2_4)

R_1 = lambda x : reps1(n1_0(x),n0_0(x),x.omid,x.zmid2,x.zmid1,x.g*1)
R_2 = lambda x : reps2(n1_1(x),n0_1(x),n2_1(x),x.tbar_1,x.o_1,x.z2_1,x.z1_1,x.g*1)
R_3 = lambda x : reps2(n1_2(x),n0_2(x),n2_2(x),x.tbar_2,x.o_2,x.z2_2,x.z1_2,x.g*1)
R_4 = lambda x : reps2(n1_3(x),n0_3(x),n2_3(x),x.tbar_3,x.o_3,x.z2_3,x.z1_3,x.g*1)


#search-space size in level-0
S_0 = lambda x : multiH(1,[n2_0(x),n2_0(x),n1_0(x),n1_0(x)])
S_1 = lambda x : multiH(x.g*1,[n2_1(x),n2_1(x),n1_1(x),n1_1(x)]) + multiH((1-x.g)/2,[(1-x.g)*w/4,(1-x.g)*w/4])
S_2 = lambda x : multiH(x.g*1,[n2_2(x),n2_2(x),n1_2(x),n1_2(x)]) + multiH((1-x.g)/4,[(1-x.g)*w/8,(1-x.g)*w/8])
S_3 = lambda x : multiH(x.g*1,[n2_3(x),n2_3(x),n1_3(x),n1_3(x)]) + multiH((1-x.g)/8,[(1-x.g)*w/16,(1-x.g)*w/16])
S_4 = lambda x : multiH(x.g*1,[n2_4(x),n2_4(x),n1_4(x),n1_4(x)]) + multiH((1-x.g)/16,[(1-x.g)*w/32,(1-x.g)*w/32])
SS_5 = lambda x : multiH(x.g*1,[n2_5(x),n2_5(x),n1_5(x),n1_5(x)]) + multiH((1-x.g)/32,[(1-x.g)*w/64,(1-x.g)*w/64])

q = lambda x : SS_5(x)*(x.a/c(suitable_diss(mem,x))-1/c(suitable_diss(mem,x)))

R_5 = lambda x : reps2(n1_4(x),n0_4(x),n2_4(x),x.tbar_4,x.o_4,x.z2_4,x.z1_4,x.g*1)+2*c(suitable_diss(mem,x))*q(x)

S_5 = lambda x : (multiH(x.g*1,[n2_5(x),n2_5(x),n1_5(x),n1_5(x)]) + multiH((1-x.g)/32,[(1-x.g)*w/64,(1-x.g)*w/64]))*x.a
S_mitm = lambda x : S_5(x)/c(suitable_diss(mem,x))

#probability that a random sum of two elements from level-1 forms a representation of any element from level-0
q_0 = lambda x : S_0(x)+R_1(x)-2*S_1(x)
q_1 = lambda x : S_1(x)+R_2(x)-2*S_2(x)
q_2 = lambda x : S_2(x)+R_3(x)-2*S_3(x)
q_3 = lambda x : S_3(x)+R_4(x)-2*S_4(x)
q_4 = lambda x : S_4(x)+R_5(x)-2*S_5(x)
q_5 = lambda x : 0

#unfiltered lists
L_mitm = lambda x : S_5(x)/2
L_5 = lambda x : 2*L_mitm(x)-x.l_5
L_4 = lambda x : 2*L_5(x)+2*q_5(x)-x.l_4
L_3 = lambda x : 2*L_4(x)+2*q_4(x)-x.l_3
L_2 = lambda x : 2*L_3(x)+2*q_3(x)-x.l_2
L_1 = lambda x : 2*L_2(x)+2*q_2(x)-x.l_1
L_0 = lambda x : 2*L_1(x)+2*q_1(x)-1

#filtered lists (for computing memory)
FL_5 = lambda x : q_5(x)+L_5(x)
FL_4 = lambda x : q_4(x)+L_4(x)
FL_3 = lambda x : q_3(x)+L_3(x)
FL_2 = lambda x : q_2(x)+L_2(x)
FL_1 = lambda x : q_1(x)+L_1(x)
FL_0 = lambda x : q_0(x)+L_0(x)

#choose magic sequence
def suitable_diss(mem,x):
    j = 0
    while S_5(x) / c(j) > mem(x):
        j = j+1
    return j

#memory for general dissection
def memdiss(x):
    x=set_vars(*x)
    i = suitable_diss(mem,x)
    mem1 = max(FL_5(x),FL_4(x),FL_3(x),FL_2(x),FL_1(x),FL_0(x),S_5(x)/c(i)) 
    return mem1

#time for general dissection
def timediss(x):
    x=set_vars(*x)
    delta = memdiss(x)/S_5(x)
    i = suitable_diss(mem,x)
    ti = max((i-c(i-1)*delta+2*delta)/(i+1),.5)
    ti = ti*S_5(x)
    return ti

def t_5(x):
    x=set_vars(*x)
    t5 = max(31*x.l_5-R_1(x)-2*R_2(x)-4*R_3(x)-8*R_4(x)-16*R_5(x),0) 
    return t5

def t_4(x):
    x=set_vars(*x)
    t4 = max(31*x.l_5+15*x.l_4-R_1(x)-2*R_2(x)-4*R_3(x)-8*R_4(x)-16*R_5(x),0)-t_5(x)
    return t4
    
def t_3(x):
    x=set_vars(*x)
    t3 = max(31*x.l_5+15*x.l_4+7*x.l_3-R_1(x)-2*R_2(x)-4*R_3(x)-8*R_4(x)-16*R_5(x),0)-t_5(x)-t_4(x)
    return t3

def t_2(x):
    x=set_vars(*x)
    t2 = max(31*x.l_5+15*x.l_4+7*x.l_3+3*x.l_2-R_1(x)-2*R_2(x)-4*R_3(x)-8*R_4(x)-16*R_5(x),0)-t_5(x)-t_4(x)-t_3(x)
    return t2

def t_1(x):
    x=set_vars(*x)
    t1 = max(31*x.l_5+15*x.l_4+7*x.l_3+3*x.l_2+x.l_1-R_1(x)-2*R_2(x)-4*R_3(x)-8*R_4(x)-16*R_5(x),0)-t_5(x)-t_4(x)-t_3(x)-t_2(x)
    return t1

def T_5(x):
    x=set_vars(*x)
    max_of_L_i = max(L_5(x),L_4(x),L_3(x),L_2(x),L_1(x),L_0(x),timediss(x)) # interpolate
    return max_of_L_i

def T_4(x):
    x=set_vars(*x)
    max_of_L_i = max(q_5(x)+L_5(x),L_4(x),L_3(x),L_2(x),L_1(x),L_0(x))
    return max_of_L_i

def T_3(x):
    x=set_vars(*x)
    max_of_L_i = max(q_4(x)+L_4(x),L_3(x),L_2(x),L_1(x),L_0(x))
    return max_of_L_i

def T_2(x):
    x=set_vars(*x)
    max_of_L_i = max(q_3(x)+L_3(x),L_2(x),L_1(x),L_0(x))
    return max_of_L_i

def T_1(x):
    x=set_vars(*x)
    max_of_L_i = max(q_2(x)+L_2(x),L_1(x),L_0(x))
    return max_of_L_i


mem = lambda x : 1

def time(x):
    x=set_vars(*x)
    max_of_L_i = max(t_1(x)+t_2(x)+t_3(x)+t_4(x)+t_5(x)+T_1(x),t_2(x)+t_3(x)+t_4(x)+t_5(x)+T_2(x),t_3(x)+t_4(x)+t_5(x)+T_3(x),t_4(x)+t_5(x)+T_4(x),t_5(x)+T_5(x))
    return max_of_L_i

def memory(x):
    x=set_vars(*x)
    max_of_FL_i = max(memdiss(x),FL_5(x),FL_4(x),FL_3(x),FL_2(x),FL_1(x),FL_0(x)) #interpolate
    return max_of_FL_i

###########################################

constraints_lwe = [
    
        
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -memory(x)+mem(x))},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : R_5(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_5(x))},

    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_1(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_2(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n0_5(x))},


    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : n1_0(x)*.5-x.omid)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.zmid1-x.zmid2+n0_0(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_1+n2_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_1+n1_1(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_1-x.z1_1+n0_1(x)/2)},

    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_2+n2_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_2+n1_2(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_2-x.z1_2+n0_2(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_3+n2_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_3+n1_3(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_3-x.z1_3+n0_3(x)/2)},
    
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.tbar_4+n2_4(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.o_4+n1_4(x)/2)},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : -x.z2_4-x.z1_4+n0_4(x)/2)},
    

    { 'type' : 'ineq',   'fun' : lwe(lambda x : x.l_5-R_5(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : 3*x.l_5+x.l_4-2*R_5(x)-R_4(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : 7*x.l_5+3*x.l_4+x.l_3-4*R_5(x)-2*R_4(x)-R_3(x))},
    { 'type' : 'ineq',   'fun' : lwe(lambda x : 15*x.l_5+7*x.l_4+3*x.l_3+x.l_2-8*R_5(x)-4*R_4(x)-2*R_3(x)-R_2(x))},
    
    #{ 'type' : 'ineq',   'fun' : lwe(lambda x : 1-n2_3(x)-n2_3(x)-n1_3(x)-n1_3(x))},

]

F6 = []
PROCESSES = 8

def opti():
    res = 100
    ress = 5000
    for j in range(10):
        start=[uniform(1,1)]+[uniform(1,1)]+[uniform(0,.5)]*5+[uniform(0,.009)]*19

        bounds=[(1,1)]+[(1,1)]+[(0,1)]*5+[(0.,1)]*19
        result = opt.minimize(time, start, 
                bounds= bounds, tol=1e-10, 
                constraints=constraints_lwe, options={'maxiter':2000})
        r = result.get('fun')
        if(r < res and r >0 and result.get('message') == 'Optimization terminated successfully'):
               res = r
               ress = result
#     if(res<memory(ress.x)):
#         print('Memory exceeds time')
#     if(res<100):
#         print((round(res,4),mem(ress.x),memory(ress.x))) 
#         F6.append((round(res,4),mem(ress.x)))
#         #print(ress.x)
#         print(t_5(ress.x),t_4(ress.x),t_3(ress.x),t_2(ress.x),t_1(ress.x))
#         Data.append((ress.x,t_5(ress.x),t_4(ress.x),t_3(ress.x),t_2(ress.x),t_1(ress.x)))
    return [res, ress]
    
def do_parallel():
    mini=[1000]
    prev=1000
    for k in range(1):
        
        candidates=[]
        with multiprocessing.Pool(PROCESSES) as pool:
            params = range(PROCESSES)
            results = [pool.apply_async(opti) for p in range(PROCESSES)]
            for i in results:

                candidates.append(i.get())
        #print(candidates)
        for i in candidates:
            if mini[0]==1000 or i[0]<mini[0]:
                mini=i
        
        if mini != prev:
            #print(k,mini)
            prev=mini
    return mini

Data = []
    
for weight in [.5]:
    w = weight
    for M in range(1,2):
        global mem
        # Specify the ammount of available memory in the line below.
        mem = lambda x : 0.34 #.02*(20-M)
        best = do_parallel()
        print("\n\n\n Best result found:\n")
        print(round(best[0],4),mem(best[1].x),memory(best[1].x), best[1].x)
        Data.append([round(best[0],4),mem(best[1].x),best[1].x])
        print('*******************************')

/tmp/ipykernel_19639/1104727675.py:113: RuntimeWarning: divide by zero encountered in double_scalars
  delta = memdiss(x)/S_5(x)
/tmp/ipykernel_19639/1104727675.py:115: RuntimeWarning: invalid value encountered in double_scalars
  ti = max((i-c(i-1)*delta+2*delta)/(i+1),.5)
/tmp/ipykernel_19639/1104727675.py:113: RuntimeWarning: invalid value encountered in double_scalars
  delta = memdiss(x)/S_5(x)
/tmp/ipykernel_19639/1104727675.py:113: RuntimeWarning: divide by zero encountered in double_scalars
  delta = memdiss(x)/S_5(x)
/tmp/ipykernel_19639/1104727675.py:115: RuntimeWarning: invalid value encountered in double_scalars
  ti = max((i-c(i-1)*delta+2*delta)/(i+1),.5)
/tmp/ipykernel_19639/1104727675.py:113: RuntimeWarning: invalid value encountered in double_scalars
  delta = memdiss(x)/S_5(x)
/tmp/ipykernel_19639/1104727675.py:113: RuntimeWarning: divide by zero encountered in double_scalars
  delta = memdiss(x)/S_5(x)
/tmp/ipykernel_19639/1104727675.py:115: RuntimeWarning: invalid v




 Best result found:

0.3228 0.34 0.32033327387847893 [1.00000000e+00 1.00000000e+00 2.79390216e-01 2.64174545e-01
 3.17890121e-01 3.16625197e-01 1.86011304e-01 1.65688512e-02
 6.65865648e-03 9.71960701e-02 4.15554342e-03 3.72693781e-03
 4.76821543e-04 4.91170181e-02 1.73247141e-03 1.72659772e-03
 2.28341788e-04 2.96175128e-02 1.21229509e-03 5.74503821e-04
 1.10757302e-04 1.62190327e-02 2.73767195e-04 8.92193452e-05
 3.69082127e-05 8.65336615e-04]
*******************************


In [13]:
#Data generated for w = .500 at depth-6

Data = [
 [0.3218,
  0.34,
  [1.00000000e+00, 1.00000000e+00, 2.75882680e-01, 2.44061658e-01,
         3.05864122e-01, 2.96706673e-01, 2.65198442e-01, 1.70235615e-02,
         7.15570853e-03, 1.01150761e-01, 3.98514562e-03, 3.79633614e-03,
         4.78926057e-04, 5.36125356e-02, 1.41464772e-03, 1.51391307e-03,
         1.92917086e-04, 3.43012187e-02, 7.86969902e-04, 6.98260453e-04,
         1.88775886e-04, 1.98961929e-02, 3.69427597e-04, 2.91758121e-04,
         1.34496156e-04, 3.96622962e-03]],
 [0.3233,
  0.32,
  [1.00000000e+00, 1.00000000e+00, 2.78523691e-01, 2.52468796e-01,
         3.01251701e-01, 3.02621384e-01, 2.14927970e-01, 1.58163713e-02,
         6.36550287e-03, 9.57731200e-02, 3.75362173e-03, 3.34770283e-03,
         3.73591008e-04, 4.91206221e-02, 1.34876826e-03, 1.32845675e-03,
         1.41929187e-04, 3.15774365e-02, 7.66142857e-04, 8.44675535e-04,
         1.23251907e-04, 1.69323151e-02, 1.72938894e-04, 5.66057924e-04,
         1.93740879e-04, 1.37660605e-03]],
 [0.3315,
  0.3,
  [1.00000000e+00, 1.00000000e+00, 2.94016893e-01, 2.69335731e-01,
         2.58538847e-01, 2.05860235e-01, 1.94065344e-01, 1.12137957e-02,
         3.43929427e-03, 7.89754688e-02, 2.49232441e-03, 1.84399698e-03,
         1.53378222e-04, 3.69026957e-02, 8.89490846e-04, 8.54865558e-04,
         9.38667223e-05, 2.28459874e-02, 6.55962473e-04, 9.33848175e-04,
         1.18089102e-04, 1.07681255e-02, 6.16749519e-04, 2.20204677e-03,
         1.41391637e-04, 7.08532360e-04]],
 [0.3402,
  0.28,
  [1.00000000e+00, 1.00000000e+00, 2.73729450e-01, 2.24118665e-01,
         2.19796699e-01, 2.08211824e-01, 3.37581870e-01, 1.22214390e-02,
         3.77693149e-03, 8.59764038e-02, 2.37094418e-03, 1.97766519e-03,
         1.73123927e-04, 4.15647755e-02, 9.32084501e-04, 1.40170929e-03,
         2.11842831e-04, 3.19933923e-02, 7.51980868e-04, 1.77794310e-03,
         6.61231160e-04, 1.78218029e-02, 9.48826525e-04, 2.57387664e-03,
         1.87156530e-03, 4.67275553e-03]],
 [0.3475,
  0.26,
  [1.00000000e+00, 1.00000000e+00, 2.48583053e-01, 1.71409666e-01,
         1.74793987e-01, 1.93712235e-01, 4.34837340e-01, 1.03520187e-02,
         2.66445581e-03, 8.28978809e-02, 1.23088786e-03, 1.05356353e-03,
         4.21380157e-05, 4.05872633e-02, 2.42366435e-04, 7.28721052e-04,
         4.43781885e-05, 4.16423504e-02, 1.81830479e-04, 9.48651841e-04,
         1.53492557e-04, 2.88995803e-02, 3.45723649e-04, 1.70456485e-03,
         1.35133834e-03, 1.19579760e-02]],
 [0.3832,
  0.24,
  [1.00000000e+00, 1.00000000e+00, 2.37343459e-01, 1.12435601e-01,
         1.27552953e-01, 1.45223139e-01, 5.26472906e-01, 7.65048015e-03,
         1.46515525e-03, 7.04098574e-02, 6.54824401e-04, 5.21371920e-04,
         1.85820016e-05, 3.25340397e-02, 2.06490675e-04, 1.36836985e-03,
         2.71945158e-04, 4.40659248e-02, 5.65376139e-04, 1.61094573e-03,
         6.21834319e-04, 3.19609391e-02, 1.02780378e-03, 2.25085105e-03,
         1.40749786e-03, 1.87753831e-02]],
 [0.4282,
  0.22,
  [1.00000000e+00, 1.00000000e+00, 2.02724088e-01, 7.00399739e-02,
         7.99866475e-02, 7.84386222e-02, 6.36346888e-01, 5.54467998e-03,
         7.76669491e-04, 6.06373362e-02, 2.84232825e-04, 2.33424851e-04,
         3.04080394e-06, 2.85513787e-02, 6.95927840e-05, 8.54599599e-04,
         1.29710601e-04, 4.81019560e-02, 2.36960837e-04, 1.55512936e-03,
         6.02322229e-04, 3.77773035e-02, 8.07618333e-04, 2.75759998e-03,
         2.17443460e-03, 2.66420729e-02]],
 [0.478,
  0.2,
  [1.00000000e+00, 1.00000000e+00, 1.98419087e-01, 5.66100141e-02,
         3.00473270e-02, 3.86231142e-02, 7.16982236e-01, 5.53560078e-03,
         8.20506846e-04, 5.79803016e-02, 3.76478679e-04, 2.71769161e-04,
         7.60510811e-06, 2.53971673e-02, 8.99932460e-05, 8.97530757e-04,
         1.83327691e-04, 4.47973956e-02, 1.95985767e-04, 1.05803159e-03,
         2.77485821e-04, 4.63038865e-02, 3.19878995e-04, 1.03516490e-03,
         5.53538287e-04, 4.17950616e-02]],
 [0.545,
  0.18,
  [1.00000000e+00, 1.00000000e+00, 1.69075253e-01, 1.95007213e-02,
         1.37159349e-02, 2.03338697e-03, 8.17332698e-01, 6.88032570e-03,
         1.18423152e-03, 6.71187970e-02, 4.89811294e-04, 3.86483886e-04,
         1.01636167e-05, 3.05339825e-02, 1.22873656e-04, 1.22842522e-03,
         1.89149624e-04, 5.23299112e-02, 3.15499350e-04, 9.58227031e-04,
         3.75346730e-04, 5.07063628e-02, 4.34751422e-04, 2.04232016e-03,
         1.01242977e-03, 4.52537119e-02]],
 [0.6364,
  0.16,
  [1.00000000e+00, 1.00000000e+00, 1.28600657e-01, 2.55895166e-03,
         2.34958066e-08, 1.63083985e-02, 8.51446516e-01, 9.45619054e-03,
         2.19776529e-03, 8.09017717e-02, 9.09368725e-04, 7.64881179e-04,
         2.83864136e-05, 3.93520323e-02, 2.67369096e-04, 1.70695075e-03,
         3.02150292e-04, 5.80196132e-02, 3.94734506e-04, 1.65940955e-03,
         3.90558173e-04, 5.35241579e-02, 6.57057238e-04, 2.24923135e-03,
         9.41496240e-04, 4.07988324e-02]]]